## Generic mapping types

The collections.abc module provides the Mapping and MutableMapping ABCs to formalize the interfaces of dict and similar types

![Alt](ContainerS.JPG)

In [5]:
from collections import abc

my_dict = {}
isinstance(my_dict, abc.Mapping)

True

Using insinstace is better than checking whether a function argument is of dict type,because then alternative mapping types can be used.

### What is hashable?

>An object is hashable if it has a hash value which never changes during its lifetime (it
needs a __hash__() method), and can be compared to other objects (it needs an
__eq__() method). Hashable objects which compare equal must have the same hash
value. […]

The atomic immutable types (str, bytes, numeric types) are all hashable. A frozen
set is always hashable, because its elements must be hashable by definition. A tuple is
hashable only if all its items are hashable. See tuples tt, tl and tf:

In [9]:
tt = (1, 2, (30, 40))
hash(tt)

8027212646858338501

In [12]:
tl = (1, 2, [30, 40])
hash(tl)

TypeError: unhashable type: 'list'

In [14]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

985328935373711578

In [15]:
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

##### Example 3-1. Examples of dict comprehensions

In [17]:
DIAL_CODES = [
(86, 'China'),
(91, 'India'),
(1, 'United States'),
(62, 'Indonesia'),
(55, 'Brazil'),
(92, 'Pakistan'),
(880, 'Bangladesh'),
(234, 'Nigeria'),
(7, 'Russia'),
(81, 'Japan')]

In [18]:
country_code = {country: code for code, country in DIAL_CODES}
country_code

{'China': 86,
 'India': 91,
 'United States': 1,
 'Indonesia': 62,
 'Brazil': 55,
 'Pakistan': 92,
 'Bangladesh': 880,
 'Nigeria': 234,
 'Russia': 7,
 'Japan': 81}

In [19]:
{code: country.upper() for country, code in country_code.items() if code < 66}

{1: 'UNITED STATES', 62: 'INDONESIA', 55: 'BRAZIL', 7: 'RUSSIA'}

## The __missing__ method

##### Example 3-7. StrKeyDict0 converts non-string keys to str on lookup

In [35]:
class StrKeyDict0(dict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()

In [36]:
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d['2']

'two'

In [37]:
d[4]

'four'

In [38]:
d[1]

KeyError: '1'

In [39]:
d.get('2')

'two'

In [40]:
d.get(1, 'N/A')

'N/A'

In [41]:
2 in d

True

In [42]:
1 in d

False

### collections.Counter

In [43]:
import collections
ct = collections.Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [44]:
ct.update('aaaaazzz')
ct

Counter({'a': 10, 'b': 2, 'r': 2, 'c': 1, 'd': 1, 'z': 3})

In [45]:
ct.most_common(2)

[('a', 10), ('z', 3)]

### Subclassing UserDict

Thanks to UserDict, StrKeyDict (Example 3-8) is actually shorter than StrKeyDict0 (Example 3-7), but it does more: it stores all keys as str, avoiding unpleasant surprises if the instance is built or updated with data containing non-string keys.

##### Example 3-8. StrKeyDict always converts non-string keys to str — on insertion, update and lookup.

In [ ]:
import collections
class StrKeyDict0(collections.UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
   #__contains__ is simpler: we can assume all stored keys are str and we can check on self.data instead of invoking self.keys() as we did in StrKeyDict0
    def __contains__(self, key):
        return str(key) in self.data
    
    # __setitem__ converts any key to a str.
    def __setitem__(self, key, item):
        self.data[str(key)] = item
  

## Immutable mappings

The mapping types provided by the standard library are all mutable, but you may need to guarantee that a user cannot change a mapping by mistake.

Since Python 3.3 the types module provides a wrapper class MappingProxyType which,
given a mapping, returns a mappingproxy instance that is a read-only but dynamic view of the original mapping. This means that updates to the original mapping can be seen in the mappingproxy, but changes cannot be made through it.

##### Example 3-9. MappingProxyType builds a read-only mappingproxy instance from a dict.

In [46]:
from types import MappingProxyType
d = {1: 'A'}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [47]:
d_proxy[1]

'A'

In [48]:
# Changes cannot be made through d_proxy.
d_proxy[2] = 'x'

TypeError: 'mappingproxy' object does not support item assignment

In [49]:
d[2] = 'B'
d_proxy # d_proxy is dynamic: any change in d is reflected

mappingproxy({1: 'A', 2: 'B'})